#  Train a GPT-2 Text-Generating Model w/ GPU For Free 

by [Max Woolf](http://minimaxir.com)

*Last updated: October 17th, 2021*

Retrain an advanced text generating neural network on any text dataset **for free on a GPU using Collaboratory** using `gpt-2-simple`!

For more about `gpt-2-simple`, you can visit [this GitHub repository](https://github.com/minimaxir/gpt-2-simple). You can also read my [blog post](https://minimaxir.com/2019/09/howto-gpt2/) for more information how to use this notebook!


To get started:

1. Copy this notebook to your Google Drive to keep it and save your changes. (File -> Save a Copy in Drive)
2. Make sure you're running the notebook in Google Chrome.
3. Run the cells below:


In [1]:
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

## GPU

Colaboratory uses either a Nvidia T4 GPU or an Nvidia K80 GPU. The T4 is slightly faster than the old K80 for training GPT-2, and has more memory allowing you to train the larger GPT-2 models and generate more text.

You can verify which GPU is active by running the cell below.

In [2]:
!nvidia-smi

Fri Nov 18 16:34:20 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Downloading GPT-2

If you're retraining a model on new text, you need to download the GPT-2 model first. 

There are three released sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.
* `774M`: the "large" model, cannot currently be finetuned with Colaboratory but can be used to generate text from the pretrained model (see later in Notebook)
* `1558M`: the "extra large", true model. Will not work if a K80/P4 GPU is attached to the notebook. (like `774M`, it cannot be finetuned).

Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by changing `model_name` in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the Colaboratory VM at `/models/<model_name>`.

This model isn't permanently saved in the Colaboratory VM; you'll have to redownload it if you want to retrain it at a later time.

In [3]:
gpt2.download_gpt2(model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, 361Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 2.69Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 435Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:19, 26.1Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 625Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 3.64Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 3.16Mit/s]


## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

Running this cell (which will only work in Colaboratory) will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code; that auth is not saved anywhere)

In [ ]:
gpt2.mount_gdrive()

## Uploading a Text File to be Trained to Colaboratory

In the Colaboratory Notebook sidebar on the left of the screen, select *Files*. From there you can upload files:

![alt text](https://i.imgur.com/TGcZT4h.png)

Upload **any smaller text file**  (<10 MB) and update the file name in the cell below, then run the cell.

In [4]:
file_name = "jvg.txt"

If your text file is larger than 10MB, it is recommended to upload that file to Google Drive first, then copy that file from Google Drive to the Colaboratory VM.

In [ ]:
gpt2.copy_file_from_gdrive(file_name)

## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [5]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=1000,
              restore_from='fresh',
              run_name='run1',
              print_every=10,
              sample_every=200,
              save_every=500
              )

Loading checkpoint models/124M/model.ckpt
Loading dataset...


100%|██████████| 1/1 [00:00<00:00,  1.75it/s]


dataset has 100058 tokens
Training...
[10 | 27.32] loss=3.42 avg=3.42
[20 | 49.70] loss=3.36 avg=3.39
[30 | 72.42] loss=3.38 avg=3.39
[40 | 94.72] loss=3.12 avg=3.32
[50 | 116.97] loss=2.90 avg=3.24
[60 | 139.43] loss=2.75 avg=3.15
[70 | 161.88] loss=2.70 avg=3.09
[80 | 184.21] loss=2.72 avg=3.04
[90 | 206.58] loss=2.71 avg=3.00
[100 | 229.00] loss=2.63 avg=2.96
[110 | 251.42] loss=2.39 avg=2.91
[120 | 273.81] loss=2.25 avg=2.85
[130 | 296.21] loss=2.13 avg=2.79
[140 | 318.55] loss=2.27 avg=2.75
[150 | 340.93] loss=1.60 avg=2.67
[160 | 363.31] loss=2.29 avg=2.64
[170 | 385.67] loss=1.64 avg=2.58
[180 | 408.06] loss=1.96 avg=2.54
[190 | 430.47] loss=1.36 avg=2.47
[200 | 452.86] loss=1.31 avg=2.41
======== SAMPLE 1 ========
t that little violets; the girl was so sweet I can’t put my hand on. I’d rather be dissatisfied.
I’ve decided to go on a suicide note until tomorrow, to see whether I can’t finish my diary. It would be a great shame if I did, because I’m busy and it would be foolish f

Instructions for updating:
Use standard file APIs to delete files with this prefix.


After the model is trained, you can copy the checkpoint folder to your own Google Drive.

If you want to download it to your personal computer, it's strongly recommended you copy it there first, then download from Google Drive. The checkpoint folder is copied as a `.rar` compressed file; you can download it and uncompress it locally.

In [ ]:
gpt2.copy_checkpoint_to_gdrive(run_name='run1')

You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.

## Load a Trained Model Checkpoint

Running the next cell will copy the `.rar` checkpoint file from your Google Drive into the Colaboratory VM.

In [ ]:
gpt2.copy_checkpoint_from_gdrive(run_name='run1')

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

In [ ]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

Loading checkpoint checkpoint/run1/model-1000
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-1000


## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

In [6]:
gpt2.generate(sess, run_name='run1')

Artificial light – the future is always beautiful – but there are also thousands of other wonderful things – people, castles, vaults, metallurgy, iron and steel, plumbing, everything imaginable, we live in – and yet there’s something so dull and drab, so peaceful and peaceful here.
Another wonderful thing is the view from the sea from shore. If I’d seen anything like it here before, it’d be a full moon today, then I would be steamroasted – I don’t know how – there was something very livelier than here, afternoons of bliss and nightmares slept in the trees, read Languedocs and Isaacs and now finally this evening – oh that first night when my eyes were dry and wet and empty mouths begging me to join them, and I woke up cold and decided to never think about anything like this again.
The most delightful hours were these, I fly high when I have nothing but work to do and pleasure to experience – Nell Lacy for the first time, oh, how beautiful it is, how pleasurable it must be to feel someth

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [8]:
gpt2.generate(sess,
              length=250,
              temperature=0.7,
              prefix="When I reflect upon Vincent's legacy, I",
              nsamples=5,
              batch_size=5
              )

When I reflect upon Vincent's legacy, I see that it’s not about him showing off (what he’s good at), it’s about showing the way. If people think about it all the time, they’ll get it wrong. I think about it all the time today – how prideful and how prosperous and prosperous Theo always was – that’s how he’d do it. He worked hard, wrote much, studied, was busy and I still think about him – today – how happy and grateful I’d been recently. I shall try to be a little bit kinder and better this week and then we’ll see how it goes.
It’s the evening before Whitsun, how we’d see him at home – I couldn’t put my hand around it so that it didn’t turn out poorly, but still I felt happy and grateful for the wonderful view from his window. Oh, that beautiful, wide-eyed child – how real that feeling was for me – oh, that sweet, pure, pure feeling that was Theo. I’ve lost him, my dear husband – oh, my loss.
Winter is approaching – and I’m
When I reflect upon Vincent's legacy, I see that it’s not abou

In [9]:
gpt2.generate(sess,
              length=250,
              temperature=0.7,
              prefix="When I reflect upon my legacy, I",
              nsamples=5,
              batch_size=5
              )

When I reflect upon my legacy, I remember the exhilarating sense of victory over my oppressor; to leave this life I feel only as if I were taken from there, I mustn’t for one moment fancy to be taken – but then I would not feel so victorious, not now, not ever. Oh, I shouldn’t have done that, I should have been afraid, I should have been so proud, it would have been so wonderful to her, it would have been so wonderful to her, it would have been so wonderful, oh, it’s all a dream, it can’t be true, it can’t be true, is it true, is it true what Shakespeare had in mind? I ask myself this evening, what wonderful thoughts and feelings I’ve had. Dreams of something good, of becoming. A person like that doesn’t work.A dreamer, on the other hand. Heists, conquers,missions,rosences,heirs, he’s just been patient enough. He’s paid a visit to Martha, he’s having a chat about her “love” on his own, hasn’t talked to her for years, and
When I reflect upon my legacy, I believe that it’s all but empty.

In [13]:
gpt2.generate(sess,
              length=250,
              temperature=0.7,
              prefix="I sometimes ponder my greatest regret.",
              nsamples=5,
              batch_size=5
              )

I sometimes ponder my greatest regret. If I had not spent so many wonderful years growing and keeping beautiful memories of my youth in Dulwich – I might have lived in Dulwich! 
It was a delightful young life at school – at least this year. Why was it so? What was the difference between the short time they went without and the whole life in which they spent it? I always asked myself this question when I was in London – and little Wight was the first to notice it. 
In London I had a very good time – at the recent Christmas party in the Museum – but there was something oppressive – almost unbearable. There was something oppressive at the table – I almost gave in to it. I am so used to it – so used to everything – it took me longer than normal. Now I have to do it again; I must do my best to be good as soon as possible.
It was a lovely day at Christmas; I sat on the floor in front of the bookcase and wrote my notes – after having spent the whole day in the country, Ihit and Ihitaka came t

For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp.

You can rerun the cells as many times as you want for even more generated texts!

In [ ]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=500,
                      temperature=0.7,
                      nsamples=100,
                      batch_size=20
                      )

In [ ]:
# may have to run twice to get file to download
files.download(gen_file)

## Generate Text From The Pretrained Model

If you want to generate text from the pretrained model, not a finetuned model, pass `model_name` to `gpt2.load_gpt2()` and `gpt2.generate()`.

This is currently the only way to generate text from the 774M or 1558M models with this notebook.

In [ ]:
model_name = "774M"

gpt2.download_gpt2(model_name=model_name)

Fetching checkpoint: 1.05Mit [00:00, 354Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 131Mit/s]                                                    
Fetching hparams.json: 1.05Mit [00:00, 279Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 3.10Git [00:23, 131Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 380Mit/s]                                                
Fetching model.ckpt.meta: 2.10Mit [00:00, 226Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 199Mit/s]                                                       


In [ ]:
sess = gpt2.start_tf_sess()

gpt2.load_gpt2(sess, model_name=model_name)

W0828 18:37:58.571830 139905369159552 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


Loading pretrained model models/774M/model.ckpt


In [ ]:
gpt2.generate(sess,
              model_name=model_name,
              prefix="The secret of life is",
              length=100,
              temperature=0.7,
              top_p=0.9,
              nsamples=5,
              batch_size=5
              )

The secret of life is that it's really easy to make it complicated," said Bill Nye, the host of the popular science show "Bill Nye the Science Guy." "And this is one of the reasons why we all need to be smarter about science, because we can't keep up with the amazing things that are going on all the time."

While Nye is correct that "everything that's going on all the time" is making the world a better place, he misses the point. This is not
The secret of life is in the rhythm of the universe. It's not a mystery. It's not a mystery to me. It's the nature of the universe. It's the beauty of the universe. It's the way the universe works. It's the way the universe is. It's the way the universe is going to work. It's the way the universe is. It's the way the universe is. It's the way the universe is. It's the way the universe is. It's the way
The secret of life is in the universe.


-

The Red Devil

It's the end of the world as we know it, and the only thing that can save us is a band of 

# Etcetera

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [ ]:
!kill -9 -1

# LICENSE

MIT License

Copyright (c) 2019 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.